In [24]:
# # dta -> csv
# import pandas as pd
# data_path = 'C:/Users/HP/Desktop/data'
# data = pd.io.stata.read_stata(data_path + '/H_DAD_w1a3.dta')
# data.to_csv(data_path + '/H_DAD_w1a3.csv') # Save as a csv

In [25]:
# csv -> df
import csv
import pandas as pd
csv_path = 'C:/Users/HP/Desktop/data/'
df = pd.read_csv(csv_path + '/H_DAD_w1a3.csv', index_col=0)
print(df.shape)

(4096, 729)


In [26]:
# show the data
df.head(10)

,prim_key,hhid,pnc,pn,r1stateid,r1iwy_d,r1iwm_d,r1lasidy,rabyear,rabmonth,...,r1cdr_mem3,r1cdr_ori3,r1cdr_jud3,r1cdr_com3,r1cdr_hom3,r1cdr_per3,r1cdr_scor3,r1raterid1,r1raterid2,r1raterid3
0,101000100040102,101000100040100,2,2,1,2019,1,224,1950,11.0,...,0.5,0.0,0.5,0.5,0.0,0.0,0.5,7049.0,7021.0,7038.0
1,101000100320109,101000100320100,9,9,1,2019,1,224,1949,9.0,...,1.0,0.5,0.5,0.5,0.5,0.0,0.5,7033.0,7038.0,7021.0
2,101000101040101,101000101040100,1,1,1,2019,1,223,1957,5.0,...,0.5,1.0,0.5,0.0,0.5,0.5,0.5,7043.0,7020.0,7049.0
3,101000101720101,101000101720100,1,1,1,2019,1,222,1957,11.0,...,2.0,2.0,3.0,2.0,3.0,3.0,3.0,7043.0,7038.0,7046.0
4,101000101890107,101000101890100,7,7,1,2019,1,222,1949,4.0,...,0.5,0.0,0.0,0.0,0.5,0.0,0.5,7049.0,7038.0,7020.0
5,101000104030101,101000104030100,1,1,1,2019,1,220,1953,2.0,...,0.5,0.0,0.0,0.0,0.0,0.0,0.5,7036.0,7035.0,7020.0
6,101000200050102,101000200050100,2,2,1,2019,2,255,1953,11.0,...,1.0,0.5,1.0,0.5,0.5,0.5,0.5,7042.0,7046.0,7036.0
7,101000200160109,101000200160100,9,9,1,2019,2,252,1938,1.0,...,1.0,1.0,1.0,0.0,0.5,0.0,0.5,7042.0,7038.0,7043.0
8,101000200740101,101000200740100,1,1,1,2019,2,252,1957,3.0,...,1.0,1.0,0.5,0.5,1.0,0.0,0.5,7049.0,7046.0,7020.0
9,101000201560108,101000201560100,8,8,1,2019,2,253,1950,NaN,...,0.0,0.5,0.5,0.0,0.0,0.0,0.5,7038.0,7036.0,7043.0


In [27]:
# for i in df.columns.tolist():
#     print(i)


In [28]:
# distribution of label
df['r1cdr_final'].value_counts()

0.5    1568
0.0     768
1.0     162
2.0      25
3.0       5
Name: r1cdr_final, dtype: int64

In [29]:
# get whatever before r1cdr_final
st = df.columns.get_loc('r1stateid')
ed = df.columns.get_loc('r1cdr_final')
print(st,ed)
new_df = df.iloc[:,0:ed+1]
print(new_df.shape)

4 703
(4096, 704)


In [30]:
# drop null data
new_df = new_df[new_df['r1cdr_final'].notnull()]
print(new_df.shape)
new_df = new_df.dropna(axis=1)
print(new_df.shape)
clean_df = new_df
print(clean_df.shape)

(2528, 704)
(2528, 504)
(2528, 504)


In [31]:
# split features and labels
y = clean_df.r1cdr_final
X = clean_df.drop(['r1cdr_final'], axis=1)
print(y.shape)
print(X.shape)

(2528,)
(2528, 503)


In [32]:
# split train_data and test_data
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
# y[y <= 0.5] = 0
# y[y > 0.5] = 1
print(y.shape)
# labelencoder = preprocessing.LabelEncoder()
# labelencoder.fit(y)
# y = labelencoder.transform(y)
# print(y)
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, test_size=0.2, random_state=0)

(2528,)


In [33]:
# SVM training
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from xgboost import XGBRegressor
my_pipeline = make_pipeline(StandardScaler(), XGBRegressor())
my_pipeline.fit(X_train, y_train)

D:\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:625: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
D:\Anaconda3\lib\site-packages\sklearn\base.py:465: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, y, **fit_params).transform(X)


Pipeline(memory=None,
     steps=[('standardscaler', StandardScaler(copy=True, with_mean=True, with_std=True)), ('xgbregressor', XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
       gamma=0, gpu_id=-1, importance_type=None,
       i...s_weight=1,
       subsample=1, tree_method='exact', validate_parameters=1,
       verbosity=None))])

In [34]:
# Test report
from sklearn.metrics import mean_squared_error
y_pred = my_pipeline.predict(X_test)
print(mean_squared_error(y_test, y_pred))

0.042482854830645476


D:\Anaconda3\lib\site-packages\sklearn\pipeline.py:331: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)
